# Testing the Monte Carlo sampler

In [27]:
using HalfIntegers, JLD2, Distributions, SL2Cfoam, DataFrames, CSV, Random

In [47]:
using JupyterFormatter
enable_autoformat()

1-element Vector{Function}:
 format_current_cell (generic function with 1 method)

The function in the cell below is the generator of bulk spins configurations that we use in the exact calculation.

In [141]:
# store self-energy spins configurations for all partial cutoffs up to cutoff
function self_energy_spins_conf(cutoff, jb::HalfInt, configs_path::String, step = half(1))

    total_number_spins_configs = Int[]

    # loop over partial cutoffs
    for pcutoff = 0:step:cutoff

        # generate a list of all spins to compute
        spins_configurations = NTuple{6,HalfInt8}[]

        for j23::HalfInt = 0:step:pcutoff,
            j24::HalfInt = 0:step:pcutoff,
            j25::HalfInt = 0:step:pcutoff,
            j34::HalfInt = 0:step:pcutoff,
            j35::HalfInt = 0:step:pcutoff,
            j45::HalfInt = 0:step:pcutoff

            # skip if computed in lower partial cutoff
            j23 <= (pcutoff - step) &&
                j24 <= (pcutoff - step) &&
                j25 <= (pcutoff - step) &&
                j34 <= (pcutoff - step) &&
                j35 <= (pcutoff - step) &&
                j45 <= (pcutoff - step) &&
                continue

            # skip if any intertwiner range empty
            r2, _ = intertwiner_range(jb, j25, j24, j23)
            r3, _ = intertwiner_range(j23, jb, j34, j35)
            r4, _ = intertwiner_range(j34, j24, jb, j45)
            r5, _ = intertwiner_range(j45, j35, j25, jb)

            isempty(r2) && continue
            isempty(r3) && continue
            isempty(r4) && continue
            isempty(r5) && continue

            # must be computed
            push!(spins_configurations, (j23, j24, j25, j34, j35, j45))

        end

        # store partial spins configurations at pctuoff
        @save "$(configs_path)/configs_pcutoff_$(twice(pcutoff)/2).jld2" spins_configurations

        total_conf = size(spins_configurations)[1]

        if (!isempty(total_number_spins_configs))
            total_conf += total_number_spins_configs[end]
        end

        println("configurations at partial cutoff = $pcutoff: $(total_conf)")
        push!(total_number_spins_configs, total_conf)

    end

    # store total spins configurations at total cutoff
    total_number_spins_configs_df =
        DataFrame(total_spins_configs = total_number_spins_configs)
    CSV.write(
        "$(configs_path)/spins_configurations_cutoff_$(twice(cutoff)/2).csv",
        total_number_spins_configs_df,
    )

end

self_energy_spins_conf (generic function with 2 methods)

In the present context, it is better to store configurations using a hash table. 

The function below is more than one order of magnitude slower than the one above (and the stored configurations occupy much more memory), but the advantage emerges in the retrieving phase

In [142]:
# store self-energy spins configurations for all partial cutoffs up to cutoff
function self_energy_spins_conf_Dict(
    cutoff,
    jb::HalfInt,
    configs_path::String,
    step = half(1),
)

    # loop over partial cutoffs
    for pcutoff = 0:step:cutoff

        # generate a Dict (hash table) of all spins to compute
        spins_configurations = Dict{Vector{HalfInt8},Int}()

        index_in_list = 0

        for j23::HalfInt8 = 0:step:pcutoff,
            j24::HalfInt8 = 0:step:pcutoff,
            j25::HalfInt8 = 0:step:pcutoff,
            j34::HalfInt8 = 0:step:pcutoff,
            j35::HalfInt8 = 0:step:pcutoff,
            j45::HalfInt8 = 0:step:pcutoff

            # skip if computed in lower partial cutoff
            j23 <= (pcutoff - step) &&
                j24 <= (pcutoff - step) &&
                j25 <= (pcutoff - step) &&
                j34 <= (pcutoff - step) &&
                j35 <= (pcutoff - step) &&
                j45 <= (pcutoff - step) &&
                continue

            # skip if any intertwiner range empty
            r2, _ = intertwiner_range(jb, j25, j24, j23)
            r3, _ = intertwiner_range(j23, jb, j34, j35)
            r4, _ = intertwiner_range(j34, j24, jb, j45)
            r5, _ = intertwiner_range(j45, j35, j25, jb)

            isempty(r2) && continue
            isempty(r3) && continue
            isempty(r4) && continue
            isempty(r5) && continue

            # must be computed
            index_in_list += 1
            spins_configurations[[j23, j24, j25, j34, j35, j45]] = index_in_list


        end

        # store partial spins configurations at pctuoff
        @save "$(configs_path)/Dict_configs_pcutoff_$(twice(pcutoff)/2).jld2" spins_configurations

        total_conf = index_in_list

        println("configurations at partial cutoff = $pcutoff: $(index_in_list)")

    end

end

self_energy_spins_conf_Dict (generic function with 2 methods)

This is the Monte Carlo sampler used in the code:

In [18]:
# store Monte Carlo self-energy spins configurations for all partial cutoffs up to cutoff
function self_energy_MC_sampling(cutoff, Nmc::Int, jb::HalfInt, MC_configs_path::String, step=half(1))

    MC_draws = Array{HalfInt8}(undef, 6, Nmc)
    draw_float_sample = Array{Float64}(undef, 1)

    # loop over partial cutoffs
    for pcutoff = step:step:cutoff

        distr = Uniform(0, Int(2 * pcutoff + 1))

        for n = 1:Nmc

            while true

                # sampling j23, j24, j25 for the 4j with spins [j23, j24, j25, jb]
                for i = 1:3
                    rand!(distr, draw_float_sample)
                    MC_draws[i, n] = half(floor(draw_float_sample[1]))
                end

                # sampling j34, j35 for the 4j with spins [j34, j35, jb, j23]
                for i = 4:5
                    rand!(distr, draw_float_sample)
                    MC_draws[i, n] = half(floor(draw_float_sample[1]))
                end

                # sampling j45 for the 4j with spins [j45, jb, j24, j34]
                for i = 6:6
                    rand!(distr, draw_float_sample)
                    MC_draws[i, n] = half(floor(draw_float_sample[1]))
                end

                # skip if computed in lower partial cutoff
                MC_draws[1, n] <= (pcutoff - step) && MC_draws[2, n] <= (pcutoff - step) &&
                    MC_draws[3, n] <= (pcutoff - step) && MC_draws[4, n] <= (pcutoff - step) &&
                    MC_draws[5, n] <= (pcutoff - step) && MC_draws[6, n] <= (pcutoff - step) && continue

                # check that 4j with spins [j45, jb, j24, j34] satisfies triangular inequalities
                r, _ = intertwiner_range(
                    MC_draws[6, n],
                    jb,
                    MC_draws[2, n],
                    MC_draws[4, n]
                )
                isempty(r) && continue

                # check that 4j with spins [j34, j35, jb, j23] satisfies triangular inequalities
                r, _ = intertwiner_range(
                    MC_draws[4, n],
                    MC_draws[5, n],
                    jb,
                    MC_draws[1, n]
                )
                isempty(r) && continue

                # check that 4j with spins [j23, j24, j25, jb] satisfies triangular inequalities
                r, _ = intertwiner_range(
                    MC_draws[1, n],
                    MC_draws[2, n],
                    MC_draws[3, n],
                    jb
                )
                isempty(r) && continue

                # check that 4j with spins [jb, j25, j35, j45] satisfies triangular inequalities
                r, _ = intertwiner_range(
                    jb,
                    MC_draws[3, n],
                    MC_draws[5, n],
                    MC_draws[6, n],
                )
                isempty(r) && continue

                # bulk spins have passed all tests -> must be computed
                break

            end

        end

        # store MC spins indices 
        @save "$(MC_configs_path)/MC_draws_pcutoff_$(twice(pcutoff)/2).jld2" MC_draws

    end

end

self_energy_MC_sampling (generic function with 2 methods)

In [158]:
CUTOFF = half(20)
JB = half(1)
PATH = "./MC_sampler_test"
mkpath(PATH)
MONTE_CARLO_ITERATIONS = 10^3

1000

## Step 1

Let's compute and store all the configurations for each partial cutoff (alias "LISTONE") using the hash table version

In [140]:
@time self_energy_spins_conf_Dict(CUTOFF, JB, PATH)

configurations at partial cutoff = 0: 0
configurations at partial cutoff = 1/2: 8
configurations at partial cutoff = 1: 73
configurations at partial cutoff = 3/2: 286
configurations at partial cutoff = 2: 758
configurations at partial cutoff = 5/2: 1728
configurations at partial cutoff = 3: 3399
configurations at partial cutoff = 7/2: 6242
configurations at partial cutoff = 4: 10564
configurations at partial cutoff = 9/2: 17164
configurations at partial cutoff = 5: 26453
configurations at partial cutoff = 11/2: 39666
configurations at partial cutoff = 6: 57306
configurations at partial cutoff = 13/2: 81164
configurations at partial cutoff = 7: 111811
configurations at partial cutoff = 15/2: 151726
configurations at partial cutoff = 8: 201512
configurations at partial cutoff = 17/2: 264480
configurations at partial cutoff = 9: 341217
configurations at partial cutoff = 19/2: 436022
configurations at partial cutoff = 10: 549406
 15.768234 seconds (82.98 M allocations: 4.243 GiB, 6.10% gc 

## Step 2

Let's compute and store all the sampled Monte Carlo configurations which DO NOT USE THE LISTONE

In [159]:
@time self_energy_MC_sampling(CUTOFF, MONTE_CARLO_ITERATIONS, JB, PATH)

  0.119369 seconds (4.65 k allocations: 328.156 KiB)


## Step 3

Now, for each pcutoff, we retrieve both the list of spins configurations computed exactly and the spins sampled with MC.

For each spin configuration sampled with MC; we save the corresponding position in the LISTONE.

In order for the sampler to be really unbiased, the position variable has to be distributed uniformly between 1 and number of configurations at each pcutoff.

In [174]:
function Test_sampler(cutoff, jb::HalfInt, configs_path::String, Nmc::Int, step = half(1))

    # loop over partial cutoffs
    for pcutoff = step:step:cutoff

        @load "$(configs_path)/Dict_configs_pcutoff_$(twice(pcutoff)/2).jld2" spins_configurations
        @load "$(configs_path)/MC_draws_pcutoff_$(twice(pcutoff)/2).jld2" MC_draws

        indices_set = Int[]

        for mc_index = 1:Nmc

            index_listone = spins_configurations[MC_draws[:, mc_index]]

            push!(indices_set, index_listone)

        end

        number_of_configs = length(spins_configurations)

        println("CONFIGURATIONS AT PCUTOFF = $pcutoff: $(number_of_configs)\n")

        expected_mean = number_of_configs / 2
        expected_skewness = 0.0
        expected_var = (number_of_configs^2 - 1) / 12

        computed_mean = mean(indices_set)
        computed_skewness = skewness(indices_set)
        computed_var = var(indices_set)

        println("expected mean = $(expected_mean)")
        println("computed mean = $(computed_mean)\n")

        println("expected skewness = $(expected_skewness)")
        println("computed skewness = $(computed_skewness)\n")

        println("expected var = $(expected_var)")
        println("computed var = $(computed_var)\n\n")

    end

end

Test_sampler (generic function with 4 methods)

In [175]:
Test_sampler(CUTOFF, JB, PATH, MONTE_CARLO_ITERATIONS)

CONFIGURATIONS AT PCUTOFF = 1/2: 8

expected mean = 4.0
computed mean = 4.484

expected skewness = 0.0
computed skewness = 0.02289757881549669

expected var = 5.25
computed var = 5.240984984984985


CONFIGURATIONS AT PCUTOFF = 1: 73

expected mean = 36.5
computed mean = 37.273

expected skewness = 0.0
computed skewness = -0.034399341500069396

expected var = 444.0
computed var = 433.9564274274275


CONFIGURATIONS AT PCUTOFF = 3/2: 286

expected mean = 143.0
computed mean = 141.323

expected skewness = 0.0
computed skewness = 0.06577430620201753

expected var = 6816.25
computed var = 6340.196867867868


CONFIGURATIONS AT PCUTOFF = 2: 758

expected mean = 379.0
computed mean = 377.178

expected skewness = 0.0
computed skewness = 0.0012836459826529884

expected var = 47880.25
computed var = 47688.32464064064


CONFIGURATIONS AT PCUTOFF = 5/2: 1728

expected mean = 864.0
computed mean = 866.256

expected skewness = 0.0
computed skewness = 0.008998608744301827

expected var = 248831.9166666